# Lab09 - BOW Representation for Image Retrieval
### CDS6334 Visual Information Processing


This lab puts you through the process of generating visual words by constructing a *visual vocabulary* based on local features such as SIFT, and proceed to create *bag of words (BOW) representation* for each image. You will also try your hand at implementing the tf-idf weighting scheme on the BOW representation, plus create some nice visualization of the visual word patches.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

## Generating visual vocabulary

To generate a good visual vocabulary that generalizes well, it is good to use a set of images, rather than just a single image. In this exercise, we will use a small set of 10 images to demonstrate this task.

**Revisting subplots.** Let's revise again on how subplots can be created using the matplotlib function `subplots` (we will be needing subplots very often in this lab). An interesting technique of cycling through all the axes objects in a figure of subplots is to use the `ravel` function which allows us to access each subplot using a single index number. This is more convenient than using row and column indices to define a subplot. Bear in mind that the single index number cycles through the subplots following [row-major order](https://en.wikipedia.org/wiki/Row-_and_column-major_order).

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(12, 3))
fig.subplots_adjust(hspace = .2, wspace=.1)    # hspace and wspace defines the horizontal and vertical gap between subplots
axs = axs.ravel()                              # "unravel" these subplots into a "vector"
for i in range(10):
    filename = "dataset/" + str(i) + ".jpg"
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    axs[i].imshow(img)
    axs[i].set_title(filename[8:])
    axs[i].set_xticks([])
    axs[i].set_yticks([])
    
#plt.savefig('wildlife.png', frameon=False, bbox_inches='tight')    
plt.show()

Occasionally, you might think this looks like a nice arrangement with some nice labeling. How can we save this figure up? Go back to the previous set of code and uncomment the line that executes the function `savefig`. You may use your own choice of filename and a number of possible image file extensions (see [here](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.savefig.html)). The parameters `frameon=False` enables transparency for the background of the figure, and `bbox_inches='tight'` creates a tight bounding box for the figure (less whitespace around it).

**Note**: You cannot call `savefig` AFTER you have called `show`, as the function `show` already dumps the graphic to your notebook.

### Local feature extraction

So, let's now get back to the feature extraction part.

Let's get the SIFT descriptors from each image.

In [ ]:
# create some lists
imgs = []
feat = []   

for i in range(10):
    filename = "dataset/" + str(i) + ".jpg"
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sift = cv2.xfeatures2d.SIFT_create(contrastThreshold=0.1)
    kps, des = sift.detectAndCompute(gray, None)
    
    imgs.append(img)
    feat.append((kps, des))     # list contains a tuple of two arrays -- keypoint, descriptor
    print('Computing SIFT for image #%d'%(i))   # some verbose

In [ ]:
# check: print number of descriptors for each image
for i in range(10):
    print(feat[i][1].shape)

Let's put the previous code for visualizing descriptors into an easy function to use:

In [ ]:
def showDescriptors(img, kps, flag=4):
    imm = np.zeros((img.shape))    
    imm = cv2.drawKeypoints(img, kps, imm, flags=flag); 
    imm = cv2.cvtColor(imm, cv2.COLOR_RGB2BGR)
    plt.imshow(imm)
    plt.show()
    
# show SIFT descriptors for a certain image
imageNum = 9
showDescriptors(imgs[imageNum], feat[imageNum][0], flag=4)

### Clustering descriptors to build visual words

To construct the visual vocabulary, we need to first compile together all the SIFT descriptors that we found from all 10 images (Note: This is considered a small amount actually. In most cases we need to build the visual vocabulary from thousands of images.)

In [ ]:
# since feat contain a list of tuples, a cool way to "unpack" or "unzip" a bunch of tuples into 
# separate lists is by using the zip function with an asterisk *
loc, des = list(zip(*feat))

# the single line above is similar to performing these two list comprehension operations
#loc = [item[0] for item in feat]
#des = [item[1] for item in feat]

# stack the lists of descriptors vertically (since descriptors are in rows)
alldes = np.vstack(des)
print(alldes.shape)

Next, we can cluster these descriptors using k-means clustering. The general rule of thumb is that the number of clusters (k) should be much lesser than the total number of descriptors (so that we don't end up with a sparse histogram)

In [ ]:
import time
from scipy.cluster.vq import kmeans, vq

# k is the number of clusters
k = 50
alldes = np.float32(alldes)      # convert to float, required by kmeans and vq functions
e0 = time.time()
codebook, distortion = kmeans(alldes, k)
code, distortion = vq(alldes, codebook)
e1 = time.time()
print("Time to build {}-cluster codebook from {} images: {} seconds".format(k,alldes.shape[0],e1-e0))

### Detour: Pickling objects

Let's detour a little to look at how to store Python objects by serializing them. We call this "pickling" (like how you pickle vegetables to keep them for a long time...). Considering that you have learned a codebook from the SIFT descriptors. This codebook can be stored away for later use, for deployment at the user/app end. If we have a huge amount of data to learn the codebook (which might be time-consuming), we should not be running this over and over, but instead, have it pickled after the heavy computations in the construction task.

In [ ]:
import pickle

pickle.dump(codebook, open("codebook.pkl", "wb") )

In [ ]:
del codebook

In [ ]:
codebook

To load back the pickle file:

In [ ]:
codebook = pickle.load( open( "codebook.pkl", "rb" ) )

In [ ]:
print(codebook.shape)
print(codebook)

Now we are ready to continue where we left off...

### Visualizing visual word patches
Let's attempt to visualize the patches that belong to a particular cluster. The cluster that is the largest (with the most descriptor patches assigned to it) can be found by this:

In [ ]:
out = np.histogram(code, bins=k)

# shows the distribution of the descriptors belonging to the k-th visual word
plt.bar(np.arange(k), out[0]), plt.show()

# find the index of the visual word that has the largest number of (descriptor) members 
print(np.argmax(out[0]))

The following code is to visualize patches that belong to a certain visual word (we select the one with the largest cluster so that we have a chance of seeing more patches). As example, we only take descriptors that belong to the first image (image 0).

In [ ]:
# select the visual word with the largest cluster, and find its descriptors that belong to image 0
im = 0
c = 5                                               # c can take values between 0 and k
code0, distortion = vq(feat[im][1], codebook)       # for ease, do VQ for descriptors of image 0
rows = np.where(code0 == c)                         # find descriptors that belong to c-th word
nrows = len(rows[0])
print("Number of descriptors of word %d in image %d: %d"%(c,im,nrows))

imPerLine = 16          # limit each line to only show a number of patches
lines = nrows//imPerLine+1
fig, axs = plt.subplots(lines, imPerLine)
fig.set_size_inches((18,4))
fig.subplots_adjust(hspace = .1, wspace=.1)
axs = axs.ravel()

plt.rcParams.update({'figure.max_open_warning': 0})

# access the keypoints of these descriptors
for r in np.arange(nrows):
    imgfeat = feat[im][0]    # get the image's features keypoints
    desnum = rows[0][r]
    winsize = np.int32(np.round(imgfeat[desnum].size))
    y = np.int32(imgfeat[rows[0][r]].pt[1])                   
    x = np.int32(imgfeat[rows[0][r]].pt[0])
    if (y+winsize > imgs[0].shape[0] or x+winsize > imgs[0].shape[1]):
         continue
    
    patch = imgs[0][y-winsize:y+winsize, x-winsize:x+winsize, :]   
    patch = cv2.resize(patch, (15, 15), interpolation = cv2.INTER_AREA)
   
    axs[r].imshow(patch, aspect='equal')
    axs[r].set_xticks([])
    axs[r].set_yticks([])

# this is to turn off the axis for remaining subplots that are unused
for r2 in np.arange(nrows,imPerLine*lines):    
    axs[r2].axis('off')
    
plt.show()

**Observation note**: If you observe that the patches for each visual word contain a large variety of views, that could mean that the number of clusters used earlier to build the vocabulary is too small. If you see that different clusters have patches that are similar, that could mean that the number of clusters is too large.

## Bag-of-words (BOW) representation

After constructing the visual vocabulary, we can now extract the Bag-of-words (BOW) representation by taking the normalized histogram of visual word occurrences.

In [ ]:
bow = list();
for i in np.arange(10):
    code, distortion = vq(feat[i][1], codebook)
    bowhist = np.histogram(code, k, density=True)
    bow.append(bowhist)

This is how the BOW histogram for image 0 looks like...

In [ ]:
plt.bar(bow[0][1][1:], bow[0][0]); 
plt.show()

Let's visualize the BOW histogram of all 10 images side-by-side (making sure y-axis is fixed)...

In [ ]:
fig, axs = plt.subplots(2, 10, figsize=(12, 3), facecolor='w', edgecolor='w')
fig.subplots_adjust(hspace = .1, wspace=.1)
axs = axs.ravel()
for i in range(10):
    filename = "dataset/" + str(i) + ".jpg"
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    axs[i].imshow(img)
    axs[i].set_title(filename[8:])
    axs[i].set_xticks([])
    axs[i].set_yticks([])
for i in range(10,20):
    axs[i].bar(bow[i-10][1][1:], bow[i-10][0]); 
    axs[i].set_xlim((0,k)), axs[i].set_ylim((0, 0.15))
    axs[i].set_xticks([]), axs[i].set_yticks([])
    
plt.show()

### Measuring distance between BOW features

In order to perform a retrieval or classification task, there must be a means to match the features extracted from these images. The cosine distance between two vectors $u$ and $v$, is defined as $$1-\frac{u \cdot v}{\|u\|\|v\|}$$ where $u \cdot v$ is the dot product between the two vectors.

Let's consider first image `'0.jpg'` as the query image. We now write a loop to find the cosine distance from the query image to the other 9 images. We also create labels to identify the relevant classes involved ('0' for tiger, '1' for turtle), and then pack both the labels and the distancs in to a tuple, appending it to a list.

In [ ]:
from scipy.spatial import distance

labels = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
results = list();
for i in np.arange(10):
    d = distance.cosine(bow[0][0], bow[i][0])
    results.append((labels[i], d))     # place the label and distance in a tuple, append to list of tuples

_ = [print(r) for r in results]

Obviously, the first image matches itself (the first image) perfectly, and hence the distance measured is 0. For all other images, a distance value between 0 and 1 is produced. The smaller this value, the closer it is to the query image, on the basis of comparing their BOW-SIFT features.

To sort the distances in the `results` tuple:

In [ ]:
sorted_results = sorted(results, key=lambda item: item[1])     # the sorting key takes the second value of the tuple
_ = [print(sr) for sr in sorted_results]

What we can observe here is that, excluding the first match (which is the same image anyway), the top 4 matches gives us images with the following labels: 0, 0, 0, 0 -- all 4 are tigers. Excellent!

**Q1**: What if another query image (try *q1.jpg* or *q2.jpg*) is used as the query image instead? Would you still be getting such perfect results? Test this out and see.
<table><tr><td><img src="q1.jpg" width="200"></td><td><img src="q2.jpg" width="200"></td></tr></table>

Complete the code in function `retrieveImgs` to output the retrieval results given an image filename. Do keep in mind that the query image needs to undergo the similar processing steps: SIFT extraction, vector quantization using the codebook constructed earlier (do not need to rebuild) and finally, matching against the images in the gallery.

In [ ]:
def retrieveImgs(filename, bow):
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # extract SIFT features

    
    # load the pickled codebook, apply vq to the features

    
    # plot BOVW histogram for the image

    
    # compute cosine distance between the query image and each of the image in the dataset
    # store the distance in the variable "results" and return it to the calling function 

    
    return results

file = 'q2.jpg'
results = retrieveImgs(file, bow)

# print out the results, sorted by distance



## Performance Metrics

### Precision and Recall
Precision and Recall are used to evaluate various types of tasks, from detection to retrieval. General precision and recall are calculated by:<br>
$$\begin{align}Precision &= \frac{\text{TP}}{\text{TP + FP}} = \frac{\text{Relevant Images}}{\text{Total Images Retrieved}}\\
Recall &= \frac{\text{TP}}{\text{TP + FN}} = \frac{\text{Relevant Images}}{\text{Total Relevant Images in Database}}\end{align}$$

However, this evaluation disregards the ranking of the images retrieved. In any retrieval tasks, the ranking indicates the quality of the retrieval algorithm. Therefore, the top-$K$ of the retrieved results should be considered, and for this we will need to calculate precision and recall at each retrieved result. 

### Precision@K

Consider the example in **Q1**, with the assumption that there are 5 relevant images, and the top 10 images that were retrieved ($K=10$) to match a tiger query image (1), are ranked as follows:
$$1, 1, 0, 1, 0, 1, 0, 0, 1, 0$$
In this case, we are only interested in $K$ number of retrieved results. 
We can calculated **Precision@K** by the same equation above but only at **K** amount of retrievals. For instance, if we were to find the Precision@1 till Precision@5 (i.e. when the top 1 till top 5 images are retrieved), we will have each of the precision as follows:
$$\begin{align} \text{Precision@1} & = \frac{1}{1} = 1 \\ 
\text{Precision@2} & = \frac{2}{2} = 1 \\ 
\text{Precision@3} & = \frac{2}{3} = 0.67 \\
\text{Precision@4} & = \frac{3}{4} = 0.75 \\
\text{Precision@5} & = \frac{3}{5} = 0.6\end{align}$$ 
This is repeated until all top-10 retrieved images are considered.

### Recall@K
Likewise, the **Recall@K** can be determined using the total relevant images in the database that is 5, as follows:
$$\begin{align} \text{Recall@1} & = \frac{1}{5} = 0.2 \\ 
\text{Recall@2} & = \frac{2}{5} = 0.4 \\ 
\text{Recall@3} & = \frac{2}{5} = 0.4 \\
\text{Recall@4} & = \frac{3}{5} = 0.6 \\
\text{Recall@5} & = \frac{3}{5} = 0.6\end{align}$$. 

These pairs of **@K** values are used to plot the Precision-Recall curve.

**Q2**: Write some functions to calculate the ranked Precision@K and Recall@K of a query image given K number of images retrieved. After that, use these functions to plot the Precision-Recall curve by generating many pairs of Precision and Recall values. Note that the Precision@K function will be useful for calculating the AP in Q3 below.

In [ ]:
#Enter your code here


### Average Precision

**Average Precision** (AP) is the average of the precision value obtained for the set of top $K$ images existing after each relevant image is retrieved. It considers the rank of the retrieved results when calculating the precision score: 
$$AP=\frac{1}{|R|}\sum_{k=1}^{K}Prec(k)\cdot rel(k)$$ 
where $R$ is the total number of relevant images, $Prec(k)$ is the precision at top $k$ documents and $rel(k)$ indicates the relevance of the $k$-th retrieved document -- 1 if relevant, 0 otherwise.

Using the same example, where 1 indicates the relevant tiger images retrieved:
$$1, 1, 0, 1, 0, 1, 0, 0, 1, 0$$
The Average Precision can be calculated as follows: 
$$\begin{align} AP & = \frac{1}{5}\biggl[\biggl(\frac{1}{1}\cdot 1 \biggr) + \biggl(\frac{2}{2}\cdot 1 \biggr) + \biggl(\frac{2}{3}\cdot 0 \biggr) + \biggl(\frac{3}{4}\cdot 1 \biggr) + \biggl(\frac{3}{5}\cdot 0 \biggr) \\ &+ \biggl(\frac{4}{6}\cdot 1 \biggr) + \biggl(\frac{4}{7}\cdot 0 \biggr) + \biggl(\frac{4}{8}\cdot 0 \biggr) + \biggl(\frac{5}{9}\cdot 1 \biggr) + \biggl(\frac{5}{10}\cdot 0 \biggr)\biggr] \\
& = \frac{1}{5}\biggl[\biggl(\frac{1}{1}\biggr) + \biggl(\frac{2}{2}\biggr) + \biggl(\frac{3}{4}\biggr) + \biggl(\frac{4}{6}\biggr) + \biggl(\frac{5}{9}\biggr) \biggr] \\ & = 0.794 \end{align}$$ 

AP provides a realistic interpretation for applications that require ranked retrieval tasks since the retrieved results that are closer to the top (or the so-called "top search hits" for web search results) carry more weight than those further lower.

**Q3**: Write a function to calculate the Average Precision (AP) of a query image.

In [ ]:
import numpy as np
from sklearn.metrics import average_precision_score
#Example
y_true = np.array([0, 0, 1, 1])
y_scores = np.array([0.1, 0.4, 0.35, 0.8])
avg = average_precision_score(y_true, y_scores)  
print(avg)

In [ ]:
#Enter your code here



### Mean Average Precision

mAP is the *average* of Average Precisions for all queries:
$$mAP = \frac{1}{Q}\sum_{q=1}^{Q}AP_q$$
where $Q$ is the total number of queries. This gives an overview of all retrieval performance when given various queries.

**Q4**: Write another function to calculate the mean Average Precision (mAP) metric, which will represent the overall performance on a full set of query images.

In [ ]:
#Enter your code here


## Additional Exercises


**Q1**: Can *local* (patch-level) **color descriptors** be used in place of SIFT descriptors to construct a Bag-of-words (BOW) feature representation? The only matter needing consideration is where your keypoints will come from. You can choose to use back the keypoints extracted from SIFT (SIFT uses a set of DoG blob detector at various scales/octaves), or you can opt to use Harris corners as keypoints. After that, obtain the descriptor from a fixed-size local window surrounding the keypoint.

In [ ]:
#Enter your code here


**Q2**: Proof by computation that the Average Precision (AP) score approximates to the area under the Precision-Recall (PR) curve. 

In [ ]:
#Enter your code here
